Topic Analysis of Review Data.

Project 1

DESCRIPTION

Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

Problem Statement: 

A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.

Domain: Amazon reviews for a leading phone brand

Analysis to be done: POS tagging, topic modeling using LDA, and topic interpretation

Content: 

Dataset: ‘K8 Reviews v0.2.csv’

Columns:

Sentiment: The sentiment against the review (4,5 star reviews are positive, 1,2 are negative)

Reviews: The main text of the review

Steps to perform:

Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.

Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.

Tasks: 

    Read the .csv file using Pandas. Take a look at the top few records.

    Normalize casings for the review text and extract the text into a list for easier manipulation.

    Tokenize the reviews using NLTKs word_tokenize function.

    Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

    For the topic model, we should  want to include only nouns.

        Find out all the POS tags that correspond to nouns.

        Limit the data to only terms with these tags.

    Lemmatize. 

        Different forms of the terms need to be treated as one.

        No need to provide POS tag to lemmatizer for now.

    Remove stopwords and punctuation (if there are any). 

    Create a topic model using LDA on the cleaned up data with 12 topics.

        Print out the top terms for each topic.

        What is the coherence of the model with the c_v metric?

    Analyze the topics through the business lens.

        Determine which of the topics can be combined.

    Create topic model using LDA with what you think is the optimal number of topics

        What is the coherence of the model?

    The business should  be able to interpret the topics.

        Name each of the identified topics.

        Create a table with the topic name and the top 10 terms in each to present to the  business.


In [1]:
import pandas as pd

In [2]:
import re, random, os, string, numpy as np

from pprint import pprint #pretty print
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [28]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [31]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [35]:
nltk.download('tagsets')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [40]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [49]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel

In [4]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
df=pd.read_csv('/content/drive/MyDrive/K8 Reviews v0.2.csv')

In [23]:
df.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [25]:
df_low=[sent.lower() for sent in df.review.values]
df_low[0]


'good but need updates and improvements'

In [29]:
df_token = [word_tokenize(sent) for sent in df.review.values]
df_token[0]

['Good', 'but', 'need', 'updates', 'and', 'improvements']

In [32]:
nltk.pos_tag(df_token[0])

[('Good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [33]:
df_pos = [nltk.pos_tag(token) for token in df_token]
df_pos[0]

[('Good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [36]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [38]:
df_noun=[]
for sent in df_pos:
    df_noun.append([token for token in sent if re.search("NN.*", token[1])])
df_noun[0]

[('updates', 'NNS'), ('improvements', 'NNS')]

In [41]:
lemm = WordNetLemmatizer()
df_lemm=[]
for sent in df_noun:
    df_lemm.append([lemm.lemmatize(word[0]) for word in sent])
df_lemm[0]

['update', 'improvement']

In [44]:
from string import punctuation
from nltk.corpus import stopwords
stop_nltk = stopwords.words("english")

In [45]:
stop_updated = stop_nltk + list(punctuation) + ["..."] + [".."]
df_stpremoved=[]
for sent in df_lemm:
    df_stpremoved.append([term for term in sent if term not in stop_updated])

In [48]:
df_stpremoved[4]

['phone', 'everThey', 'phone', 'problem', 'amazon', 'phone', 'amazon']

In [52]:

id2word = corpora.Dictionary(df_stpremoved)
texts = df_stpremoved
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[200])

[(512, 1), (513, 1), (514, 1), (515, 1)]


In [53]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

In [54]:
pprint(lda_model.print_topics())

[(0,
  '0.131*"mobile" + 0.065*"service" + 0.025*"handset" + 0.019*"center" + '
  '0.018*"device" + 0.017*"Lenovo" + 0.016*"Superb" + 0.013*"Excellent" + '
  '0.012*"centre" + 0.010*"version"'),
 (1,
  '0.122*"camera" + 0.078*"phone" + 0.061*"quality" + 0.037*"price" + '
  '0.028*"feature" + 0.021*"battery" + 0.018*"performance" + 0.017*"mode" + '
  '0.016*"range" + 0.016*"Camera"'),
 (2,
  '0.019*"speaker" + 0.014*"item" + 0.010*"expectation" + 0.010*"facility" + '
  '0.009*"weight" + 0.007*"se" + 0.007*"par" + 0.007*"ka" + 0.007*"hi" + '
  '0.006*"half"'),
 (3,
  '0.114*"Lenovo" + 0.094*"note" + 0.045*"K8" + 0.037*"k8" + 0.026*"Note" + '
  '0.017*"phone" + 0.015*"box" + 0.013*"feature" + 0.013*"option" + '
  '0.011*"call"'),
 (4,
  '0.230*"product" + 0.087*"Nice" + 0.065*"phone" + 0.036*"Awesome" + '
  '0.032*"mobile" + 0.025*"delivery" + 0.017*"Amazon" + 0.014*"price" + '
  '0.007*"experience" + 0.007*"Thanks"'),
 (5,
  '0.049*"performance" + 0.046*"charger" + 0.042*"update" + 0.041

In [55]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df_stpremoved, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.49429914449377493


Looking at the topics and each terms following can be combined -
Topic 2 and 5 possibly talks about 'pricing'
Topic 4, 6 and 10 closely talks about 'battery related issues'
Topic 3 and 11 vaguely talks about 'performance'

In [56]:
# Build LDA model
lda_model6 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

In [58]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model6, texts=df_stpremoved, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5394251469904519


In [59]:
x = lda_model6.show_topics(formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

In [60]:
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

0::['phone', 'service', 'money', 'call', 'issue', 'Lenovo', 'network', 'device', 'problem', 'waste']
1::['phone', 'camera', 'quality', 'price', 'feature', 'Camera', 'battery', 'performance', 'mode', 'range']
2::['hai', 'Everything', 'ho', 'NOT', 'IS', 'PHONE', 'bhi', 'bill', 'AND', 'MOBILE']
3::['note', 'Lenovo', 'phone', 'K8', 'k8', 'speaker', 'Note', 'music', 'Bluetooth', 'box']
4::['product', 'phone', 'mobile', 'Nice', 'Amazon', 'battery', 'Awesome', 'Worst', 'Lenovo', 'delivery']
5::['phone', 'battery', 'problem', 'issue', 'Battery', 'day', 'heating', 'performance', 'backup', 'hour']



In [ ]:
#Topic 1 can be Phone Issues
#Topic 2 can be Phone range
#Topic 3 can be Phone features
#Topic 4 can be Phone Accessories
#Topic 5 can be Phone Review
#Topic 6 can be Phone problems